# CS 7643 Assignment 2 Part 1:  Implement and train a network on CIFAR-10

Convolutional Neural Networks (CNNs) are one of the major advancements in
computer vision over the past decade. In this assignment, you will complete
a simple CNN architecture from scratch and learn how to implement CNNs
with PyTorch, one of the most commonly used deep learning frameworks.
You will also run different experiments on imbalanced datasets to evaluate
your model and techniques to deal with imbalanced data.

### Load Extensions
Before getting started we need to run some standard code to set up our environment. You'll need to execute this code again each time you start the notebook.

First, run this cell to load the [autoreload](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload) extension. This enables us to modify `.py` source files and reintegrate them into the notebook, ensuring a smooth editing and debugging experience.


In [1]:
%load_ext autoreload
%autoreload 2

### Google Colab Setup
Next we need to run a few commands to set up our environment on Google Colab. If you are running this notebook on a local machine you can skip this section.

Run the following cell to mount your Google Drive. Follow the link, sign in to your Google account (the same account you used to store this notebook!).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Now remember the path in your Google Drive where you uploaded this notebook, fill it in below. If all functions properly, executing the next cell should display the filenames from the assignment:

```
['CS7643-Assignment2-1.ipynb', 'cs7643', 'data', 'configs', 'models', 'optimizer', 'tests']
```

In [3]:
import os

# TODO: Fill in the Google Drive path where you uploaded assignment1
# Example: If you create a Fall2023 folder and put all the files under A1 folder, then 'Fall2023/A1'
GOOGLE_DRIVE_PATH_POST_MYDRIVE = None
GOOGLE_DRIVE_PATH = os.path.join('/content', 'drive', 'MyDrive', GOOGLE_DRIVE_PATH_POST_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'NoneType'

In [ ]:
import sys

sys.path.append(GOOGLE_DRIVE_PATH)

### Local Setup OR Google Drive
Run the cell below regardless of whether you are using google drive or local setup.

In [2]:
# if running locally set GOOGLE PATH
import sys
if 'google.colab' in sys.modules:
  print(f'Running in google colab. Our path is `{GOOGLE_DRIVE_PATH}`')
else:
  GOOGLE_DRIVE_PATH = '.'
  print('Running locally.')

Running locally.


After successfully identifying the path to this assignment, execute the following cell to enable us to import from the `.py` files of this assignment. If it works correctly, it should print the message:

```
Roger that from conv_classifier.py!
Roger that from softmax_ce.py!
Roger that from linear.py!
Roger that from relu.py!
Roger that from max_pool.py!
Roger that from convolution.py!

Roger that from _base_optimizer.py!
Roger that from sgd.py!
```

as well as the last edit time for the files `_conv_classifier.py`, `convolution.py`, `linear.py`, `max_pool.py`, `relu.py`, `softmax_ce.py`, `_base_optimizer.py`, and `sgd.py`. You may have to try running the cell twice if the first time fails.

In [3]:
import numpy as np
import math

from cs7643.env_prob import say_hello_do_you_copy

say_hello_do_you_copy(GOOGLE_DRIVE_PATH)


---------- Modules ------------------
Roger that from conv_classifier.py!
Roger that from softmax_ce.py!
Roger that from linear.py!
Roger that from relu.py!
Roger that from max_pool.py!
Roger that from convolution.py!
conv_classifier.py last edited on Mon Sep  8 20:15:29 2025
softmax_ce.py last edited on Mon Sep  8 20:15:29 2025
linear.py last edited on Tue Sep 16 20:00:13 2025
relu.py last edited on Thu Sep 11 16:32:36 2025
max_pool.py last edited on Mon Sep 15 17:54:03 2025
convolution.py last edited on Mon Sep  8 20:15:29 2025

---------- Optimizer ------------------
Roger that from _base_optimizer.py!
Roger that from sgd.py!
_base_optimizer.py last edited on Mon Sep  8 20:15:29 2025
sgd.py last edited on Mon Sep  8 20:15:29 2025


# Load the CIFAR10 dataset
Data loading is the very first step of any machine learning pipelines. Run the following cell to download the CIFAR10 dataset.

In [4]:
from cs7643.cifar10 import CIFAR10

train_ds = CIFAR10(GOOGLE_DRIVE_PATH + '/data/cifar10', download=True, train=True)
test_ds = CIFAR10(GOOGLE_DRIVE_PATH + '/data/cifar10', download=True, train=False)

Files already downloaded and verified
Files already downloaded and verified


# Implementing CNN from Scratch
You will work in `./part1-convnet` for this part of the assignment. **Note that
vectorization is not a requirement for this part of the assignment**.

## Module Implementation
You will now learn how to build CNN from scratch. Typically, a convolutional
neural network is composed of several different modules and these modules
work together to make the network effective. For each module, you will
implement a forward pass (computing forwarding results) and a backward
pass (computing gradients). Therefore, your tasks are as follows:

Follow the instructions in the code to complete each module in `./modules`. Specifically, modules to be implemented are 2D convolution, 2D
Max Pooling, ReLU, and Linear. These will be the building blocks of
the full network. The file `./modules/conv_classifier.py` ties each of the
aforementioned modules together and is the subject of the next section. Run the following cells to test your implementations.

In [9]:
#Let's test your implementation of Linear
!pytest -s {GOOGLE_DRIVE_PATH.replace(' ', '\ ') + '/tests/test_linear.py'}

<string>:1: SyntaxWarning: invalid escape sequence '\ '


============================= test session starts =============================
platform win32 -- Python 3.12.0, pytest-8.4.1, pluggy-1.5.0
rootdir: C:\Users\julie\CSE_7643_DL\DL_PS2\part1-convnet
plugins: anyio-4.7.0
collected 2 items

tests\test_linear.py F.

================================== FAILURES ===================================
__________________________ TestLinear.test_backward ___________________________

self = <tests.test_linear.TestLinear testMethod=test_backward>

    def test_backward(self):
        x = np.random.randn(10, 2, 3)
        w = np.random.randn(6, 5)
        b = np.random.randn(5)
        dout = np.random.randn(10, 5)
    
        dx_num = eval_numerical_gradient_array(lambda x: self._affine_forward(x, w, b), x, dout)
        dw_num = eval_numerical_gradient_array(lambda w: self._affine_forward(x, w, b), w, dout)
        db_num = eval_numerical_gradient_array(lambda b: self._affine_forward(x, w, b), b, dout)
    
        dx, dw, db = self._affine_backward

In [23]:
#Let's test your implementation of Max Pooling
!pytest -s {GOOGLE_DRIVE_PATH.replace(' ', '\ ') + '/tests/test_maxpool.py'}

<string>:1: SyntaxWarning: invalid escape sequence '\ '


============================= test session starts =============================
platform win32 -- Python 3.12.0, pytest-8.4.1, pluggy-1.5.0
rootdir: C:\Users\julie\CSE_7643_DL\DL_PS2\part1-convnet
plugins: anyio-4.7.0
collected 2 items

tests\test_maxpool.py dx_num:  [[ 0.          0.        ]
 [-1.20371295  0.        ]]
..

============================== 2 passed in 0.29s ==============================


In [6]:
#Let's test your implementation of RELU
!pytest -s {GOOGLE_DRIVE_PATH.replace(' ', '\ ') + '/tests/test_relu.py'}

<string>:1: SyntaxWarning: invalid escape sequence '\ '


============================= test session starts =============================
platform win32 -- Python 3.12.0, pytest-8.4.1, pluggy-1.5.0
rootdir: C:\Users\julie\CSE_7643_DL\DL_PS2\part1-convnet
plugins: anyio-4.7.0
collected 2 items

tests\test_relu.py ..

============================== 2 passed in 0.11s ==============================


In [ ]:
#Let's test your implementation of Conv2D
!pytest -s {GOOGLE_DRIVE_PATH.replace(' ', '\ ') + '/tests/test_conv.py'}

## Network Implementation
After finishing each module, it's time to put things together to form a real
convolutional neural network.

Follow the instructions in the code to complete a CNN network in `./modules/conv_classifier.py`. The network is constructed by a list of module definitions in order and should handle both forward and backward communication between modules.

# Optimizer
You have implemented a simple SGD optimizer in assignment-1. In practice, it is common to use a momentum term in SGD for better convergence. Specifically, we introduce a new velocity term $v_t$ and the update rule is as follows:

$$
v_t = \beta v_{t-1} - \eta \frac{\partial L}{\partial w} \\
w = w + v_t
$$

where $\beta$ denotes the momentum coefficient and $\eta$ denotes the learning rate.

Follow the instructions in the code to complete SGD with momentum
in `./optimizer/sgd.py`. **Hint**: you will need to store and use the velocity
from the previous iteration of SGD to compute the new gradient for
the current iteration. Feel free to add member variable(s) to achieve
this.

In [ ]:
# Test your SGD implementation
!pytest -s {GOOGLE_DRIVE_PATH.replace(' ', '\ ') + '/tests/test_sgd.py'}

# Experiments
Now, you have completed coding the entire training process. It’s time to play with your model a little. If you're already attempted to train your implementation, you might have noticed that it is extremely slow. Therefore, we only want to deliberately overfit the model with a small portion of data to verify whether the model is learning something or not.

You will train a small CNN with only 50 samples in CIFAR-10 dataset. The script will make a plot on the training data only and **be sure to include the plot in your report**. Your final accuracy should be slightly under `0.9` with the given network in the script. To start the training, execute the next cell.

In [ ]:
import matplotlib.pyplot as plt

from modules import ConvNet
from optimizer import SGD
from cs7643.solver import Solver
from data import get_CIFAR10_data

root = GOOGLE_DRIVE_PATH + '/data/cifar10/cifar-10-batches-py'
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data(root)

print(f'Number of training samples {len(X_train)}')

model_list = [dict(type='Conv2D', in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2),
              dict(type='ReLU'),
              dict(type='MaxPooling', kernel_size=2, stride=2),
              dict(type='Linear', in_dim=8192, out_dim=10)]
criterion = dict(type='SoftmaxCrossEntropy')
model = ConvNet(model_list, criterion)
optimizer = SGD(model, learning_rate=0.0001, reg=0.001, momentum=0.9)

trainer = Solver()

loss_history, train_acc_history = trainer.train(
    X_train[:50], y_train[:50], model, batch_size=10, num_epochs=10,
    verbose=True, optimizer=optimizer
    )

plt.plot(train_acc_history)
plt.legend(['train'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.savefig(GOOGLE_DRIVE_PATH + '/train.png')

# Submit Your Work
After completing the notebook for this assignment (`assignment2_1.ipynb`), run the following cell to create a `.zip` file for you to download and to upload to Gradescope.

**Please MANUALLY SAVE `*.py` files before executing the following cell:**

In [ ]:
from cs7643.submit import make_a2_1_submission

make_a2_1_submission(GOOGLE_DRIVE_PATH)